# Tests for Batch Norm

In [43]:
import numpy as np

import PyNetwork
import PyNetworkCPU

import pyopencl as cl
import pyopencl.array as cl_array

# from PyNetwork.layers import BatchNormGrads
# from PyNetworkCPU.layers import BatchNormGrads as BatchNormGradsCPU

In [44]:
input_shape = (100,)
output_nodes = (100, )

platform = cl.get_platforms()
devices = platform[0].get_devices()
context = cl.Context(devices)
queue = cl.CommandQueue(context)

layer_cpu = PyNetworkCPU.layers.BatchNorm()
layer_cpu.build(input_shape)

layer_gpu = PyNetwork.layers.BatchNorm()
layer_gpu.build(context, queue, input_shape)


# Test Predict

In [45]:
from PyNetwork import utils

In [46]:
n_datapoints = 20
z_in = np.random.rand(n_datapoints, *input_shape).astype(np.float32) - 0.5
z_in_gpu = cl_array.to_device(queue, z_in)

a_true, z_out_true = layer_cpu.predict(z_in, output_only=False)
a_true_gpu, z_out_true_gpu = layer_gpu.predict(z_in_gpu, output_only=False)

# Test the result
np.testing.assert_almost_equal(a_true, a_true_gpu.get(), decimal=6)
np.testing.assert_almost_equal(z_out_true, z_out_true_gpu.get(), decimal=6)

# Test Backprop

In [47]:
g_prime = np.random.rand(n_datapoints, *input_shape).astype(np.float32)
new_delta = np.random.rand(n_datapoints, *output_nodes).astype(np.float32)
prev_z = np.random.rand(n_datapoints, *input_shape).astype(np.float32)

g_prime_gpu = cl_array.to_device(queue, g_prime)
new_delta_gpu = cl_array.to_device(queue, new_delta)
prev_z_gpu = cl_array.to_device(queue, prev_z)

delta_true = layer_cpu.get_delta_backprop_(g_prime, new_delta, prev_z)
delta_true_gpu = layer_gpu.get_delta_backprop_(g_prime_gpu, new_delta_gpu, prev_z_gpu)

# Test the result
np.testing.assert_almost_equal(delta_true, delta_true_gpu.get(), decimal=5)

# Test Weight Gradients

In [48]:
delta = np.random.rand(n_datapoints, *input_shape).astype(np.float32)
prev_z = np.random.rand(n_datapoints, *output_nodes).astype(np.float32)

delta_gpu = cl_array.to_device(queue, delta)
prev_z_gpu = cl_array.to_device(queue, prev_z)

delta_grad_true, w_grad_true = layer_cpu.get_weight_grad_(delta, prev_z)
delta_grad_true_gpu, w_grad_true_gpu = layer_gpu.get_weight_grad_(delta_gpu, prev_z_gpu)

# Test the result
np.testing.assert_almost_equal(delta_grad_true, delta_grad_true_gpu.get(), decimal=5)
np.testing.assert_almost_equal(w_grad_true, w_grad_true_gpu.get(), decimal=5)